See threads here - 
https://www.kaggle.com/c/feedback-prize-2021/discussion/299169, regarding some observations
https://www.kaggle.com/c/feedback-prize-2021/discussion/296879, initial impetus, including linked papers.

Thanks to @cdeotte for a lot of his contributions as well. 

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy

In [ ]:
#globals 
train_texts, valid, oof = None, None, None

In [ ]:
import os, pandas as pd, numpy as np
import spacy
import re

from nltk.corpus import brown
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk import pos_tag_sents

#pretty handy
#https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(*args):
    tds = "text-align:left; border-left: 1px solid #000;border-right: 1px solid #000;"
    html_str=''
    wd = int(100/len(args))
    for df,title in args:
        html_str+=f'<td style="{tds}min-width: {wd}%;width: {wd}%;">'
        html_str+=f'<b>{title}</b><br>{df}'
        html_str+='</td>'
    #print(html_str)
    display_html("<table style='border: 1px solid black;'>"+html_str+"</table>",raw=True)
    
    

#stolen from https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533 
def init(validcsv='../input/tfbasic/valid.csv', oofcsv='../input/tfbasic/oof.csv', feedbackdir="../input/feedback-prize-2021/train"):
    global train_texts, valid, oof
    train_names, train_texts = [], []
    print("Loading train texts..", end = " ")
    for f in list(os.listdir(feedbackdir)):
        train_names.append(f.replace('.txt', ''))
        train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
    train_texts = pd.DataFrame({'id': train_names, 'text': train_texts})    
    print(".. done.")
    print("Loading valid and oof..", end = " ")
    train_texts.head()
    #import dataset with valid (subset of train, holdout for CV) and oof (predictions for valid set)
    valid = pd.read_csv('../input/tfbasic/valid.csv')
    oof = pd.read_csv('../input/tfbasic/oof.csv')
    print(".. done.")
    oof['oof_discourse_id'] = range(len(oof)) #i find this more clear than index
    oof["pred_len"] = oof["predictionstring"].apply(lambda x: len(x.split()))
    valid["pred_len"] = valid["predictionstring"].apply(lambda x: len(x.split()))
    oof["start"] = oof["predictionstring"].apply(lambda x: int(x.split()[0]))
    oof["end"] = oof["predictionstring"].apply(lambda x: int(x.split()[-1]))
    valid["start"] = valid["predictionstring"].apply(lambda x: int(x.split()[0]))
    valid["end"] = valid["predictionstring"].apply(lambda x: int(x.split()[-1]))
    valid["pred_len"] = valid["predictionstring"].apply(lambda x: len(x.split()))
    print(f"Train Texts Shape {np.shape(train_texts)}  OOF Shape {np.shape(oof)} Valid Shape {np.shape(valid)} ")
    return oof, valid, train_texts

                        
import matplotlib.pyplot as plt
import matplotlib

import io
import base64

def fig_to_html(fig):
    img = io.BytesIO()
    fig.savefig(img, format='png',
                bbox_inches='tight')
    img.seek(0)
    plt.close(fig)
    base = base64.b64encode(img.getvalue())
    encoded = fig_to_base64(fig)
    my_html = '<img src="data:image/png;base64, {}">'.format(encoded.decode('utf-8'))
    return my_html


def show_lengths(oof, valid):
    def get_html(train, dtn):
        #print("get_html:", train.keys())
        df = train.groupby(dtn).mean()[['pred_len', 'start', 'end']]
        #display(df)
        df.insert(0, "count", train.groupby(dtn).count()['id'])
        df.insert(0, "perc", df['count']/len(train))
        #print(df.keys())
        df.loc['Column_TotalOrMean'] = df.sum(numeric_only=True, axis=0)
        df.loc['Column_TotalOrMean']['pred_len'] = df['pred_len'].mean()
        df.loc['Column_TotalOrMean']['start'] = df['start'].mean()
        df.loc['Column_TotalOrMean']['end'] = df['end'].mean()
        return df.to_html()
    display_side_by_side((get_html(oof, "class"), "oof class perc, counts and avg lengths "), (get_html(valid, "discourse_type"), "valid type perc, counts and avg lengths"))

#just used for visualization, should be mostly accurate.  let me know if you see an error or improvement.
#tricky going from pi -> discourse start/end because of regex limitations
def ps_to_pos(txt, ts, ps):  
    tl = 0
    for i in range(ps):
        tl = tl + len(ts[i])
    tl = tl + ps - 4
    if tl < 0:
        tl = 0
    words_re = re.compile(re.escape(ts[ps]))
    sub = txt[tl:]
    #print(piv, tl, txt, sub, words_re)
    #print('----')
    span = words_re.search(sub).span()
    return span[0]+tl, span[1]+tl


def get_span(txt, piv):
    pi = [int(x) for x in piv.split()]
    ts = txt.split()
    ss = ps_to_pos(txt, ts, pi[0])    
    se = ps_to_pos(txt, ts, pi[-1])    
    return (ss[0],se[1])

def gen_spacy(idv, oof_filterlist = None, valid_filterlist = None):
    global valid, oof
    print(idv)
    txt = train_texts.query(f"id == '{idv}'").iloc[0]['text']
    t = valid.query(f"id == '{idv}'")    
    ents = []
    for i,r in t.iterrows():
        if valid_filterlist is not None and not r['discourse_id'] in valid_filterlist:
            continue
        s = int(r['discourse_start'])
        e = int(r['discourse_end'])
        l = r['discourse_type']
        ents.append({"start": s, "end": e, "label": l})
    tret = { "text": txt,"ents": ents,"title": None}
    o = oof.query(f"id == '{idv}'")
    ts = txt.split()
    ents = []
    for i,r in o.iterrows():  
        if oof_filterlist is not None and not r['oof_discourse_id'] in oof_filterlist:
            continue
        s,e = get_span(txt, r['predictionstring'])
        l = r['class']
        ents.append({"start": s, "end": e, "label": l})
    oret = { "text": txt,"ents": ents,"title": None}
    return tret, oret


from IPython.display import display_html
from itertools import chain,cycle

    
def get_id_from_dis(lst, comp, idc = 'pred_id'):
    #print(lst[0:10])
    df = pd.DataFrame(np.column_stack([lst]), columns = [idc])
    mrg = pd.merge(comp, df, on=idc)['id'].value_counts().reset_index()
    #print(mrg[0:10])
    mrg.columns = ['id', 'value_count']
    return mrg

# CODE FROM : Rob Mulla @robikscube
# https://www.kaggle.com/robikscube/student-writing-competition-twitch
def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]

#from https://www.kaggle.com/cdeotte/tensorflow-longformer-ner-cv-0-633, with mods
def score_feedback_comp(pred_df, gt_df, output = False):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_id', 'discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df['discourse_id']
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    if output:
        print("joined")
        display(joined)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()[['gt_id','pred_id']].values
    if output:
        print("tp_pred_ids")
        display(tp_pred_ids)
    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids[:,1:]]
    if output:
        print("fp_pred_ids")
        display(fp_pred_ids)
    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]
    if output:
        print("unmatched_gt_ids")
        display(unmatched_gt_ids)
    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    print(my_f1_score)
    score = (my_f1_score, 
             tp_pred_ids, get_id_from_dis(tp_pred_ids[:,1:], pred_df), 
             fp_pred_ids, get_id_from_dis(fp_pred_ids, pred_df), 
             unmatched_gt_ids, get_id_from_dis(unmatched_gt_ids, gt_df,'discourse_id')
            )

    #print(score[IF1])

    display_side_by_side((score[TPID].describe().to_html(), "True Positive ID Counts"), 
                         (score[FPID].describe().to_html(), "False Positive ID Counts"),  
                         (score[UNID].describe().to_html(), "Unmatched ID Counts"))
    return score

import random
def color_generator(number_of_colors):
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(number_of_colors)]
    return color

#cols = color_generator(10)
#_ = [print(i,cols[i]) for i in range(10)]

def display_txt(idvs,oof_filterlist = None, valid_filterlist = None ):
    for idv in idvs:
        spans = gen_spacy(idv, oof_filterlist, valid_filterlist)
        #print(spans[1], spans[0])
        options = {"ents": ['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement',
           'Counterclaim', 'Rebuttal'],
               "colors": {"Lead": "#A7D8FE", 'Position': "#D2B4DE", 'Evidence': "#D7BDE2", 'Claim': "#F5B041", 
                          'Concluding Statement': "#E88049",'Counterclaim': "#7FB3D5", 'Rebuttal': "#82E0AA"}}
        r1 = spacy.displacy.render(spans[1], options = options, manual=True, jupyter = False, style="ent")
        r2 = spacy.displacy.render(spans[0],options = options, manual=True, jupyter = False, style="ent")
        display_side_by_side((r1,"oof"), (r2, "valid"))

#used to index into score
IF1=0
TPDI=1 #tp (discourse_id, oof_discourse_id)
TPID=2 #tp (text id, value count) sorted by value counts descending
FPDI=3 #fp (oof_discourse_id)
FPID=4 #fp (text id, value count) sorted by value counts descending
UNDI=5 #unmatched (discourse_id)
UNID=6 #unmatched (text id, value count) sorted by value counts descending


In [ ]:
#valid.csv is small (10%?) subset of train.csv that wasn't used for training and held out for cross validation
#oof.csv is "Out Of Fold" predictions from the model on valid -> id,class, predictionstring
#the goal of this notebook is to understanding differences between valid and oof to see how the model is succeeding and failing
#average 

oof, valid, train_texts = init(validcsv='../input/tfbasic/valid.csv', oofcsv='../input/tfbasic/oof.csv', feedbackdir="../input/feedback-prize-2021/train")

#I don't include CV score here, assuming this is already done elsewhere
score = score_feedback_comp(oof, valid, output = False)



In [ ]:
#print(oof.keys(), valid.keys())

show_lengths(oof, valid)

print("Top 3 False Positives")
#FP
idvs = score[FPID][0:3]['id']
display_txt(idvs, oof_filterlist = score[FPDI])
print("\n\n\n\n\n\nTop 3 Unmatched")
#unmatched
idvs = score[UNID][0:10]['id']
display_txt(idvs, valid_filterlist = score[UNDI])
print("\n\n\n\n\n\nTop 3 True Positive")
#unmatched
idvs = score[TPID][0:3]['id']
display_txt(idvs, oof_filterlist = score[TPDI])